# Extract Custom Fields from Your File

This notebook demonstrates how to use analyzers to extract custom fields from your input files.

## Prerequisites
1. Ensure Azure AI service is configured following [steps](../README.md#configure-azure-ai-service-resource)
2. Install the required packages to run the sample.

In [ ]:
%pip install -r ../requirements.txt

## Analyzer Templates

Below is a collection of analyzer templates designed to extract fields from various input file types.

These templates are highly customizable, allowing you to modify them to suit your specific needs. For additional verified templates from Microsoft, please visit [here](../analyzer_templates/README.md).

In [ ]:
extraction_templates = {
    "invoice":            ('../analyzer_templates/invoice.json',         '../data/invoice.pdf'            ),
    "chart":              ('../analyzer_templates/image_chart.json',     '../data/pieChart.jpg'           ),
    "call_recording":     ('../analyzer_templates/call_recording_analytics.json', '../data/callCenterRecording.mp3'),
    "conversation_audio": ('../analyzer_templates/conversational_audio_analytics.json', '../data/callCenterRecording.mp3'),
    "marketing_video":    ('../analyzer_templates/marketing_video.json', '../data/video.mp4'              ),
    "PostCallAnalyticsOnText":  ('../CU-Demo-Assets/analyzers/PostCallAnalyticsOnText.json', '../CU-Demo-Assets/en-us-conversation-transcripts/sample0 - Insurance - Customer calls about purchasing life insurance.wav.results.txt'),
    "PostCallAnalyticsCustomized": ('../CU-Demo-Assets/analyzers/PostCallAnalyticsCustomized.json', '../CU-Demo-Assets/en-US-mono-audio/sample4 - Healthcare - Customer calls about new policy quote.wav'),
    "PostCallAnalyticsCustomizedDE": ('../CU-Demo-Assets/analyzers/PostCallAnalyticsWithGermanResult.json', '../CU-Demo-Assets/de-DE-mono-audio/sample4 - Healthcare - Customer calls about new policy quote- German.wav'),
    "ImageTextAnalysis": ('../CU-Demo-Assets/analyzers/imageTextAnalysis.json', '../CU-Demo-Assets/en-us-images-w-text/Designer.png'),
    "MultiLanguageDemo": ('../CU-Demo-Assets/analyzers/MultiLanguageDemo.json', '../CU-Demo-Assets/pt-BR-mono-audio/sample4 - Healthcare - Customer calls about new policy quote - Brazilian.wav'),
    "MultiLanguageDemoOnText":  ('../CU-Demo-Assets/analyzers/MultiLanguageDemoOnText.json', '../CU-Demo-Assets/pt-BR-conversation-transcripts/sample6 - Financial - Customer calls about transferring funds - Brazilian.wav.results.txt'),
    "PostCallAnalyticsCustomizedMultiLanguageMoreFields":  ('../CU-Demo-Assets/analyzers/PostCallAnalyticsCustomizedMultiLanguageMoreFields.json', '../CU-Demo-Assets/de-DE-mono-audio/sample4 - Healthcare - Customer calls about new policy quote- German.wav'),
    "ACSTesting":  ('../CU-Demo-Assets/analyzers/ACSTesting.json', '../CU-Demo-Assets/en-US-mono-audio/sample4 - Healthcare - Customer calls about new policy quote.wav'),
    "CarInsuranceCallAnalysis":  ('../CU-Demo-Assets/analyzers/CarInsuranceCallAnalysis.json', '../CU-Demo-Assets/en-US-mono-audio/Car Insurance Call.wav'),
    "StoryAnalysis": ('../CU-Demo-Assets/analyzers/HTML-ShortStoryAnalyzer.json', '../CU-Demo-Assets/en-us-text-examples/extended_short_story.html'),
    "StoryAnalysisWObject": ('../CU-Demo-Assets/analyzers/HTML-ShortStoryAnalyzerWObject.json', '../CU-Demo-Assets/en-us-text-examples/extended_short_story.html'), # Not yet supported
}

Specify the analyzer template you want to use and provide a name for the analyzer to be created based on the template.

In [ ]:
import uuid

ANALYZER_TEMPLATE = "CarInsuranceCallAnalysis"
ANALYZER_ID = "cu-demo-" + str(uuid.uuid4())

(analyzer_template_path, analyzer_sample_file_path) = extraction_templates[ANALYZER_TEMPLATE]

## Create Azure AI Content Understanding Client

> The [AzureContentUnderstandingClient](../python/content_understanding_client.py) is a utility class containing functions to interact with the Content Understanding API. Before the official release of the Content Understanding SDK, it can be regarded as a lightweight SDK.


In [ ]:
import logging
import json
import os
import sys
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

AZURE_AI_ENDPOINT = os.getenv("AZURE_AI_ENDPOINT")
AZURE_AI_API_VERSION = os.getenv("AZURE_AI_API_VERSION", "2024-12-01-preview")

# Add the parent directory to the path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from python.content_understanding_client import AzureContentUnderstandingClient

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_ENDPOINT,
    api_version=AZURE_AI_API_VERSION,
    token_provider=token_provider,
    x_ms_useragent="azure-ai-content-understanding-python/field_extraction", # This header is used for sample usage telemetry, please comment out this line if you want to opt out.
)

## List all analyzers created in your resource

After the analyzer is successfully created, we can use it to analyze our input files.

In [ ]:
response = client.get_all_analyzers()
print(f"Number of analyzers in your resource: {len(response['value'])}")
print(f"First 3 analyzer details: {json.dumps(response['value'], indent=2)}")

## Create Analyzer from the Template

In [ ]:
response = client.begin_create_analyzer(ANALYZER_ID, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)

print(json.dumps(result, indent=2))

# Extract Conversation (WEBVTT) from either Batch Transcription JSON or Content Understanding JSON

## Extract Fields Using the Analyzer

After the analyzer is successfully created, we can use it to analyze our input files.

In [ ]:
response = client.begin_analyze(ANALYZER_ID, file_location=analyzer_sample_file_path)
result = client.poll_result(response)

print(json.dumps(result, indent=2))

## Clean Up
Optionally, delete the sample analyzer from your resource. In typical usage scenarios, you would analyze multiple files using the same analyzer.

In [ ]:
client.delete_analyzer(ANALYZER_ID)